In [21]:
#bulk insert api reference: https://elasticsearch-py.readthedocs.io/en/7.x/helpers.html
from elasticsearch import Elasticsearch
host = "http://localhost:9200"
user = "elastic"
password = "password"
client = Elasticsearch(host, basic_auth=(user, password))

In [22]:
import json
#json file is a list of json instead of standard json format
data = [json.loads(line) for line in open("./shakespeare.json", "rb")]
types = []
for i in range(0, len(data), 2):
    type = data[i]["index"].get("_type")
    if type not in types:
        types.append(type)
print(types)


['act', 'scene', 'line']


In [24]:
index_name = "shakespeare"
client.indices.delete(index=index_name, ignore=[400, 404])
client.indices.create(index=index_name, ignore=400)

/tmp/ipykernel_6329/72972983.py:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.delete(index=index_name, ignore=[400, 404])
/tmp/ipykernel_6329/72972983.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client.indices.delete(index=index_name, ignore=[400, 404])
/tmp/ipykernel_6329/72972983.py:3: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  client.indices.create(index=index_name, ignore=400)
/tmp/ipykernel_6329/72972983.py:3: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/ela

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'shakespeare'})

In [25]:
from elasticsearch import helpers
def generate_doc(file_path, type):
    data = [json.loads(line) for line in open(file_path, "rb")]
    for i in range(0, len(data), 2):
        data[i].get("index").get("_type") == type
        yield {
        "_index": "shakespeare-{}".format(type),
        "_id": data[i].get("index").get("_id"),
        "_source": data[i+1]
            }

helpers.bulk(client, actions=generate_doc("./shakespeare.json", "act"))
helpers.bulk(client, actions=generate_doc("./shakespeare.json", "line"))
helpers.bulk(client, actions=generate_doc("./shakespeare.json", "scene"))


/tmp/ipykernel_6329/1790052615.py:12: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  helpers.bulk(client, actions=generate_doc("./shakespeare.json", ""))


(111396, [])